# Pipeline

In [ ]:
import scipy.sparse as sps
from Builder import Builder
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample, split_train_in_two_percentage_global_sample_k_fold
from Recommenders.XGboostRecommender import XGboostRecommender
import pickle
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from tqdm import tqdm


saved_urm_path = 'saved_models'

URM = sps.load_npz(f'{saved_urm_path}/URM.npz')
URM_train = sps.load_npz(f'{saved_urm_path}/URM_train.npz')
URM_val = sps.load_npz(f'{saved_urm_path}/URM_val.npz')
URM_test = sps.load_npz(f'{saved_urm_path}/URM_test.npz')
eval = EvaluatorHoldout(URM_val, cutoff_list=[10])
etest = EvaluatorHoldout(URM_test, cutoff_list=[10])

# Train

In [ ]:
load_model_path = f'{saved_urm_path}/train'
xgboost = XGboostRecommender(URM_train, URM_val, load_model_path=load_model_path, categorical_method=None)

In [ ]:
hs={'n_estimators': 22, 'learning_rate': 0.20846188322980433, 'reg_alpha': 0.0001673396406118902, 'reg_lambda': 0.26168707544035263, 'max_depth': 7, 'max_leaves': 15, 'grow_policy': 'lossguide'}

In [ ]:
xgboost.fit(**hs, cutoff=35)

In [ ]:
xgboost.plot_importance()

In [ ]:
result_df, _ = etest.evaluateRecommender(xgboost)
result_df['MAP'][10]

# Train Val

In [ ]:
load_model_path = f'{saved_urm_path}/train_val'
xgboost_final = XGboostRecommender(URM_train + URM_val, URM_test, load_model_path=load_model_path, categorical_method=None)

In [ ]:
xgboost_final.fit(**hs, cutoff=35)

In [ ]:
xgboost_final.plot_importance()

In [ ]:
result_df, _ = etest.evaluateRecommender(xgboost_final)
result_df['MAP'][10]

In [ ]:
xgboost_final.prepare_to_submit(URM, cutoff=35, load_model_path='saved_models/train_val_test')

In [ ]:
builder = Builder()
builder.preprocess()
df_submit = builder.prepare_sub(xgboost_final)
builder.create_sub(df_submit, file_name='xgboost_final')